In [ ]:
import glob
import scipy.stats as stats
from scipy.optimize import curve_fit
import datetime
from datetime import timezone, timedelta
from dateutil.parser import parse
import numpy as np
import os
import pathlib

import pandas as pd
import geopandas as gpd

import xarray as xr

import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.io.shapereader as shpreader
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import colors
import matplotlib.lines as mlines
import matplotlib.ticker as ticker
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
from matplotlib.colors import LinearSegmentedColormap,ListedColormap
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import matplotlib.patheffects as path_effects
#plt.rc('mathtext', fontset="cm")
%matplotlib inline
import seaborn as sns

from pyproj import Transformer
from shapely.geometry import Polygon
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score, r2_score

# Data preparation

In [ ]:
###############################################################################
# User inputs
###############################################################################
REPORT_FOLDER = 'C:/.../reports/' # Folder of hail reports in CSV format
REPORT_FOLDER_4km = 'C:/.../reports_4km/' # Folder of hail reports in CSV format
T0 = '20150502' # Start of time range #20180709
T1 = '20220903' # End of time range #20210802
DPI = 300 # DPI for the plots
OUTPUT_FOLDER = 'C:/.../plots' # output folder for plots, a subfolder will be created for plots
EXCLUDE_SENSORS = ['SchulhausHASLE']
EXCLUDE_DATES = ['2020-06-15','2020-06-16','2020-06-17','-06-18']
EXCLUDE_SENSORS_2 = ['RsiLUGANO','SportivoRIVERA','IdaMEDEGLIA']
EXCLUDE_DATES_2 = ['2019-03-11','2019-03-17','2019-03-18','2020-06-18']
EXCLUDE_SENSORS_3 = ['SportivoRIVERA']
EXCLUDE_DATES_3 = ['2019-09-05']
EXCLUDE_SENSORS_4 = ['UniBE']
EXCLUDE_DATES_4 = ['2021-06-29']
#EVENT_MIN_TIME = 600 #Minimum time separating two hail events in seconds
MIN_CZC = 35 #Minimum max reflectivity at sensor location to filter out non hail impacts
SENSOR_SURFACE = 0.25**2 * np.pi # sensor surface in m2
SENSOR_SURFACE_KM = SENSOR_SURFACE * 10**-6
folder_dt_analysis = foldername + 'sensitivity_analysis/'

In [ ]:
def ceil_dt(dt, delta = 5):
    delta = datetime.timedelta(minutes=delta)
    return dt + (datetime.datetime.min - dt) % delta

# Create output folder
foldername = OUTPUT_FOLDER + '/plots_hail_{:s}_{:s}/'.format(T0,T1)
if not os.path.exists(foldername):
    os.makedirs(foldername)
    
# Convert to datetime for comparison
T0 = datetime.datetime.strptime(T0,'%Y%m%d')
T1 = datetime.datetime.strptime(T1,'%Y%m%d')

def concate_report(folder):
    # Read all reports in time range
    all_files = sorted(glob.glob(folder + '*.txt'))
    reports_data = list()
    for f in all_files:
        date = datetime.datetime.strptime(os.path.basename(f).split('_')[-1].split('.')[0], '%Y%m%d')
        if date >= T0 and date <= T1:
            reports_data.append(pd.read_csv(f))
    reports_data = pd.concat(reports_data)
    return reports_data

reports_4km = concate_report(REPORT_FOLDER_4km)
reports_10km = concate_report(REPORT_FOLDER)
reports_10km = reports_10km[['sensorName','Time','maxpoh_rad10','maxmeshs_rad10','maxczc_rad10']]

reports_data = pd.merge(reports_4km, reports_10km, how='inner', on=['sensorName','Time'])

# Add day and closest radar timestamp to df
reports_data['days'] = [a[0:10] for a in reports_data['Time'].values]
reports_data['months'] = [a[0:7] for a in reports_data['Time'].values]
reports_data['years'] = [a[0:4] for a in reports_data['Time'].values]
reports_data['rad_timestamp'] = [ceil_dt(parse(a), 5) for a in reports_data['Time'].values]
reports_data['digits'] = reports_data['digits'].astype(np.int64)

reports_data = reports_data[~((reports_data['sensorName'].isin(EXCLUDE_SENSORS)) &
                              (reports_data['days'].isin(EXCLUDE_DATES)))]

reports_data = reports_data[~((reports_data['sensorName'].isin(EXCLUDE_SENSORS_2)) &
                              (reports_data['days'].isin(EXCLUDE_DATES_2)))]

reports_data = reports_data[~((reports_data['sensorName'].isin(EXCLUDE_SENSORS_3)) &
                              (reports_data['days'].isin(EXCLUDE_DATES_3)))]

reports_data = reports_data[~((reports_data['sensorName'].isin(EXCLUDE_SENSORS_4)) &
                              (reports_data['days'].isin(EXCLUDE_DATES_4)))]

path_install_date = 'C:/.../Hail Sensors/'
file_install_date = 'sensors_install_dates.csv'

install_data = pd.read_csv(path_install_date + file_install_date, sep=';',dtype={'sensorName':str, 'installDate': str})
reports_data = pd.merge(reports_data, install_data, how="inner", on = 'sensorName')
reports_data['installDate'] = pd.to_datetime(reports_data['installDate'])

#inProj = 'epsg:2056' # CH1903+ / LV95, see https://epsg.io/2056
inProj = 'epsg:21781' # CH1903 / LV03, see https://epsg.io/21781
outProj = 'epsg:4326' # WGS84, see https://epsg.io/4326

transformer = Transformer.from_crs(inProj, outProj)
coords = transformer.transform(reports_data['CHY'], reports_data['CHX'])

reports_data['Lat'] = coords[0].tolist()
reports_data['Lon'] = coords[1].tolist()

conditions = [(reports_data['CHX'] < 140000),((reports_data['CHX'] > 140000) & (reports_data['CHX'] < 220000)),
    (reports_data['CHX'] > 220000)]
val = ['Ticino', 'Napf', 'Jura']
reports_data['region'] = np.select(conditions, val)


sensors_list = reports_data.groupby('sensorName').agg({'Lon': 'first', 'Lat': 'first', 'CHX': 'first', 'CHY': 'first',
                                                      'installDate': 'first', 'region': 'first'}).reset_index()

In [ ]:
def make_sensors_filtered(reports_data, filter_field, field_value):
    # Filtered data
    sensors_filtered = reports_data[(reports_data['status'] == 0) & (reports_data['diameter'] > 0)
                                    & (reports_data['kin_energy'] > 0) & (reports_data['digits'] > 50)
                                    & (reports_data['calib_factor'] != 0) & (reports_data['calib_factor'].notnull())
                                   & (reports_data[filter_field] >= field_value)]

    # Remove duplicates (same sensor and same Time)
    sensors_filtered = sensors_filtered.drop_duplicates(subset=['sensorName', 'Time'], keep='first')

    temp = sensors_filtered.groupby(['sensorName','days'])
    sensors_filtered = sensors_filtered.set_index(['sensorName', 'days'])
    sensors_filtered['daily_count_by_sensor'] = temp.size()
    sensors_filtered['day_sensor_ID'] = sensors_filtered.groupby(['sensorName','days']).ngroup()
    sensors_filtered = sensors_filtered.reset_index()
    sensors_filtered['Day_D_max'] = sensors_filtered.groupby(['day_sensor_ID'])['diameter'].transform('max')

    sensors_filtered['diameter_est'] = np.power(sensors_filtered['kin_energy']/0.0462285,1/4)*10
    sensors_filtered['vt_est'] = 13.77 * np.power(sensors_filtered['diameter_est']/10,0.5)

    sensors_filtered['m_est'] = 2 * sensors_filtered['kin_energy']/(sensors_filtered['vt_est']**2)

    sensors_filtered['d_H20'] = np.where(sensors_filtered['diameter_est']<22.1,
                                                    np.power(sensors_filtered['kin_energy']/0.0107,1/4.59)*10,
                                                    np.power(sensors_filtered['kin_energy']/0.0243,1/3.55)*10)

    sensors_filtered['m_H20'] = 0.372 * np.power(sensors_filtered['d_H20']/10,2.69)

    sensors_filtered['area_H20'] = 0.62 * np.power(sensors_filtered['d_H20']/10,1.96)
    sensors_filtered['area_est'] = np.pi/4 * np.power(sensors_filtered['diameter_est']/10,2)

    sensors_filtered['vt_H20'] = np.where(sensors_filtered['d_H20']<15.9,
                                                    np.power(7.6*sensors_filtered['d_H20']/10, 0.89),
                                                    np.power(8.4*sensors_filtered['d_H20']/10, 0.67))

    sensors_filtered['KE_H20'] = 0.5 * sensors_filtered['m_H20'] * sensors_filtered['vt_H20']**2

    sensors_filtered['ratio_H20'] = sensors_filtered['d_H20']/sensors_filtered['diameter_est']

    sensors_filtered = sensors_filtered.sort_values(['sensorName', 'Time'], ascending=[True, True])
    sensors_filtered['Time2'] = pd.to_datetime(sensors_filtered['Time'])
    sensors_filtered['Time2'] = sensors_filtered['Time2'].dt.tz_localize(timezone.utc)

    conditions2 = [(sensors_filtered['kin_energy'] <= 0.07),
                   ((sensors_filtered['kin_energy'] > 0.07) & (sensors_filtered['kin_energy'] <= 0.13)),
                   ((sensors_filtered['kin_energy'] > 0.13) & (sensors_filtered['kin_energy'] <= 0.26)),
                   ((sensors_filtered['kin_energy'] > 0.26) & (sensors_filtered['kin_energy'] <= 0.67)),
                   ((sensors_filtered['kin_energy'] > 0.67) & (sensors_filtered['kin_energy'] <= 3.8)),
                  (sensors_filtered['kin_energy'] > 3.8)]

    val2 = [0.064, 0.18, 0.332, 0.5, 0.87, 0.994]
    sensors_filtered['dead_time'] = np.select(conditions2, val2)
    
    return sensors_filtered

In [ ]:
def make_sensors_data(sensors_data, EVENT_MIN_TIME):
    
    data = sensors_filtered.copy()
    data['Time_diff'] = data['Time2'].diff().dt.total_seconds()
    data['Time_diff_p'] = np.where((data['Time_diff']<0) | (data['Time_diff'].abs()/EVENT_MIN_TIME >=1),1,0)
    data = data.assign(Event_ID=data.Time_diff_p.cumsum())
    data['Event_start'] = data.groupby(['Event_ID'])['Time2'].transform('min')
    data['Event_end'] = data.groupby(['Event_ID'])['Time2'].transform('max')
    data['Event_duration'] = (data['Event_end'] - data['Event_start']).dt.total_seconds()
    data['Time_0'] = (data['Time2'] - data['Event_start']).dt.total_seconds()
    data['Time_rel'] = data['Time_0']/data['Event_duration']
    temp = data.groupby(['Event_ID'])
    data = data.set_index(['Event_ID'])
    data['Event_hits'] = temp.size()
    data['Event_density'] = data['Event_hits']/SENSOR_SURFACE
    data = data.reset_index()
    data.drop(['Time_diff_p'], axis=1, inplace=True)
    data['Event_tot_dead_time'] = data.groupby(['Event_ID'])['dead_time'].transform('sum')

    data['Event_hit_rate'] = np.where(data['Event_duration']>0,
                                                        data['Event_hits']/data['Event_duration'],
                                                        0)

    data['Event_hit_rate_adj2'] = np.where(data['Event_duration']>0,
                                                        data['Event_hits']/(data['Event_duration'] - data['Event_tot_dead_time']),
                                                        0)
    data['Event_hits_adj2'] = np.where(data['Event_duration']>0,(data['Event_hit_rate_adj2'] * data['Event_duration']).round(0).astype(int),
                                                    1)
    data['Event_hits_diff2'] = data['Event_hits_adj2'] - data['Event_hits']

    data['dt'] = data['Time_diff'].abs()
    data['dt_next'] = data['Time2'].diff(periods=-1).dt.total_seconds().abs()
    data['dt'] = data['dt'].where(data['dt']<=EVENT_MIN_TIME, other=np.NaN)
    data['dt_next'] = data['dt_next'].where(data['dt']<=EVENT_MIN_TIME, other=np.NaN)
    data['Event_dt_avg'] = data.groupby(['Event_ID'])['dt'].transform(lambda x: x.mean(skipna=True))
    data['Event_dt_min'] = data.groupby(['Event_ID'])['dt'].transform(lambda x: x.min(skipna=True))
    data['Event_inst_hit_rate'] = 1/data['dt']
    data['Event_inst_hit_rate_max'] = data.groupby(['Event_ID'])['Event_inst_hit_rate'].transform(lambda x: x.max(skipna=True))
    data['Event_inst_hit_rate_avg'] = data.groupby(['Event_ID'])['Event_inst_hit_rate'].transform(lambda x: x.mean(skipna=True))
    data['Event_inst_hit_rate_std'] = data.groupby(['Event_ID'])['Event_inst_hit_rate'].transform(lambda x: x.std(skipna=True))
    data['dt'].fillna(0, inplace=True)
    data['dt_next'].fillna(0, inplace=True)
    data['Event_inst_hit_rate'].fillna(0, inplace=True)

    data['Occurrences'] = 1
    roll_calc = data.groupby('Event_ID').rolling('10s', on='Time2', center=True)['Occurrences'].sum()
    roll_calc2 = data.groupby('Event_ID').rolling('30s', on='Time2', center=True)['Occurrences'].sum()
    data = data.set_index(["Event_ID", "Time2"])
    data["10s_hit_rate"] = roll_calc
    data["30s_hit_rate"] = roll_calc2
    data=data.reset_index()
    data['Event_10s_hit_rate_max'] = data.groupby(['Event_ID'])['10s_hit_rate'].transform('max')
    data['Event_30s_hit_rate_max'] = data.groupby(['Event_ID'])['30s_hit_rate'].transform('max')
    data['Event_10s_hit_rate_avg'] = data.groupby(['Event_ID'])['10s_hit_rate'].transform('mean')
    data['Event_30s_hit_rate_avg'] = data.groupby(['Event_ID'])['30s_hit_rate'].transform('mean')
    data['Event_10s_hit_rate_std'] = data.groupby(['Event_ID'])['10s_hit_rate'].transform('std')
    data['Event_30s_hit_rate_std'] = data.groupby(['Event_ID'])['30s_hit_rate'].transform('std')
    
    conditions = [(data['Event_hits'] == 1),
                   ((data['Event_hits'] > 1) & (data['Event_hits'] <= 5)),
                   ((data['Event_hits'] > 5) & (data['Event_hits'] <= 25)),
                  (data['Event_hits'] > 25)]

    val = ['1 impact', '2-5 impacts', '6-25 impacts', '> 25 impacts']
    data['Event_hits_group'] = np.select(conditions, val)

    
    data['Event_KEdensity_tot'] = data.groupby(['Event_ID'])['kin_energy'].transform('sum')/SENSOR_SURFACE
    data['Event_KEFlux_tot'] = data['Event_KEdensity_tot']/data['Event_duration']
    data['Event_KE_max'] = data.groupby(['Event_ID'])['kin_energy'].transform('max')
    data['Event_D_max'] = data.groupby(['Event_ID'])['diameter'].transform('max')
    data['Event_D_max_vicinity'] = (data['Event_D_max'] - 5.06)/0.493
    
    data['Event_count'] = data.groupby(['Event_hits'])['Event_ID'].transform('nunique')
    data['meanCZCrad10_ev'] = data.groupby(['Event_ID'])['maxczc_rad10'].transform('mean')
    data['meanCZCrad4_ev'] = data.groupby(['Event_ID'])['maxczc_rad4'].transform('mean')
    data['meanCZCpix_ev'] = data.groupby(['Event_ID'])['czc_sensorpix'].transform('mean')

    return data

In [ ]:
def make_hail_events_full(data):
    
    hail_events = data.groupby('Event_ID')['Event_hits','Event_hits_group','Event_count','Event_hits_adj2','Event_duration','Event_D_max','daily_count_by_sensor',
                                           'Event_dt_min', 'Event_dt_avg','Event_tot_dead_time','Event_inst_hit_rate_avg','Event_10s_hit_rate_avg','Event_30s_hit_rate_avg',
                                           'Event_inst_hit_rate_std','Event_10s_hit_rate_std','Event_30s_hit_rate_std',
                                           'Event_hit_rate','Event_inst_hit_rate_max','Event_10s_hit_rate_max','Event_30s_hit_rate_max',
                                           'Event_hit_rate_adj2', 'Event_hits_diff2',
                                           'Event_density','Event_start','Event_end',
                                           'sensorName','installDate','days',
                                           'meanCZCpix_ev','meanCZCrad4_ev','meanCZCrad10_ev',
                                           'Event_KEdensity_tot','Event_KEFlux_tot',
                                           'Event_KE_max','Event_D_max_vicinity','region'].agg('min')

    idx = data.groupby(['Event_ID'])['diameter'].transform(max) == data['diameter']
    max_d = data.loc[idx,['Event_ID','Time_0']].set_index('Event_ID')
    max_d2 = max_d.groupby('Event_ID').agg('min')
    max_d_ = data.loc[idx,['Event_ID','Time_rel']].set_index('Event_ID')
    max_d_2 = max_d_.groupby('Event_ID').agg('min')
    df3 = pd.merge(hail_events, max_d2, how="inner", on = 'Event_ID')
    hail_events_full = pd.merge(df3, max_d_2, how="inner", on = 'Event_ID')
    
    return hail_events_full

In [ ]:
def make_sensors_sel(data, data2, HIT_TH, HIT_TH_UP):
    hail_events_sel = data.loc[(data['Event_hits'] >= HIT_TH) & (data['Event_hits'] <= HIT_TH_UP)]
    sensors_ev_sel = data2.loc[(data2['Event_hits'] >= HIT_TH) & (data2['Event_hits'] <= HIT_TH_UP)]
    
    return hail_events_sel, sensors_ev_sel

# Data and methods

## Figure 1

In [ ]:
# First, let's load SwissTopo Relief! This will be our map background
# read relief and data
da_relief = xr.open_rasterio('relief_georef_clipped_swiss.tif')

# Compute the lon/lat coordinates with rasterio.warp.transform
ny, nx = len(da_relief['y']), len(da_relief['x'])
x, y = np.meshgrid(da_relief['x'], da_relief['y'])
# Rasterio works with 1D arrays
transformer = Transformer.from_crs(da_relief.crs, outProj)
lat, lon = transformer.transform(x.flatten(), y.flatten())
lon = np.asarray(lon).reshape((ny, nx))-0.01
lat = np.asarray(lat).reshape((ny, nx))
da_relief.coords['lon'] = (('y', 'x'), lon)
da_relief.coords['lat'] = (('y', 'x'), lat)

# get band
da_relief = da_relief.isel(band=0, drop=True)
da_relief = da_relief.where(da_relief > 1, drop=True)

# let's load Lakes from Swisstopo
import geopandas
gdf_lakes = geopandas.read_file("swiss-lakes-maps.json")

# cities and urban areas
import cartopy.io.shapereader as shapereader

cities = shapereader.natural_earth(resolution='10m',
                                      category='cultural',
                                      name='populated_places')
gdf_cities = geopandas.read_file(cities)
swiss_cities = gdf_cities.loc[gdf_cities['NAMEASCII'].isin(['Geneva','Zurich','Lausanne',
                                      'Basel','Bern','Lugano',
                                      'Luzern','Winterthur'])]

gdf_urban2 = geopandas.read_file('data/ne_10m_urban_areas_landscan.shp')

cmap = ListedColormap(["tab:orange", "tab:green", "tab:red"])

In [ ]:
# Sensor locations
sns.set(style='ticks', context='paper')
cm = 1/2.54  # centimeters in inches
fig = plt.figure(figsize=(11, 10), dpi=DPI)
prj = ccrs.AlbersEqualArea(8.222665776, 46.800663464)
prj2 = ccrs.PlateCarree()
ax = plt.axes(projection=prj)

fs = 14
sp = 100

CH = [5.9, 10.7, 45.7, 47.9] #45.0
ZRH = [7.9,9.3,46.95,47.7]
Ticino2 = [8.83, 9, 46.00, 46.12]

extent = CH
ax.set_extent(extent)#,prj2)

# Sensor data

sensors = ax.scatter(sensors_list['Lon'],sensors_list['Lat'],
            marker='o',
            s=sp-20,
            zorder=4, c=sensors_list['installDate'].dt.year,
           edgecolors='black', linewidth=0.25, transform=ccrs.Geodetic(),cmap=cmap)

handles, labels = sensors.legend_elements()
labels = ['Spring 2018', 'Spring 2019', 'Spring 2020']
legend1 = ax.legend(handles, labels, loc="upper left", prop={'size': fs}, framealpha=1, markerscale=2, title='Time of installation')
legend1.get_title().set_fontsize(fs)
ax.add_artist(legend1)

#add cities
transform = prj2._as_mpl_transform(ax)
for x, y, label in zip(swiss_cities.geometry.x, swiss_cities.geometry.y, swiss_cities.NAMEASCII):
    ax.annotate(label, xy=(x, y), xycoords=transform, zorder=4,
                fontsize=fs, xytext=(6, 4), textcoords="offset points")

#add sensor location labels
ax.text(7.2,47.25,'Jura', zorder=5,
        ha='center', va='center', size=fs+1, color ='black', weight='bold', transform=ccrs.PlateCarree())

ax.text(8.15,46.8,'Napf', zorder=5,
        ha='center', va='center', size=fs+1, color ='black', weight='bold', transform=ccrs.PlateCarree())

ax.text(9.25,46.15,'Ticino', zorder=5,
        ha='center', va='center', size=fs+1, color ='black', weight='bold', transform=ccrs.PlateCarree())

#add urban areas
gdf_urban2.geometry.plot(ax=ax,transform=prj2,zorder=1.4,linewidth=0,alpha=0.5, facecolor='red')

def rect_from_bound(xmin, xmax, ymin, ymax):
    """Returns list of (x,y)'s for a rectangle"""
    xs = [xmax, xmin, xmin, xmax, xmax]
    ys = [ymax, ymax, ymin, ymin, ymax]
    return [(x, y) for x, y in zip(xs, ys)]

# request data for use by geopandas
resolution = '10m'
category = 'cultural'
name = 'admin_0_countries'

shpfilename = shapereader.natural_earth(resolution, category, name)
df = geopandas.read_file(shpfilename)

# get geometry of a country
poly = [df.loc[df['ADMIN'] == 'Switzerland']['geometry'].values[0]]
ax.add_geometries(poly, crs=prj2, facecolor='none', edgecolor='black', linewidth=0.2)

pad1 = .1  #padding, degrees unit
pad2 = .1 #.4
exts = [poly[0].bounds[0] - pad1, poly[0].bounds[2] + pad1, poly[0].bounds[1] - pad2, poly[0].bounds[3] + pad1];
ax.set_extent(exts, crs=prj2)

# make a mask polygon by polygon's difference operation
# base polygon is a rectangle, another polygon is simplified switzerland
msk = Polygon(rect_from_bound(*exts)).difference(poly[0].simplify(0.01))
msk_stm  = prj.project_geometry(msk, prj2)  # project geometry to the projection

# plot the mask using semi-transparency (alpha=0.65) on the masked-out portion
ax.add_geometries(msk_stm, prj, zorder=1.5, facecolor='white', edgecolor='none')
    
# add Relief
da_relief.plot(ax=ax, x='lon', y='lat', cmap="Greys_r",
               norm=colors.Normalize(vmin=110, vmax=255), add_colorbar=False, transform=prj2)
# add lakes
gdf_lakes.plot(ax=ax, edgecolor='none', color="cornflowerblue", transform=prj2)

gl = ax.gridlines(crs=prj2, draw_labels=True, linewidth=0.25, color='gray', alpha=0.4, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xlabel_style = {'size': fs}
gl.ylabel_style = {'size': fs}
gl.xformatter = LongitudeFormatter()
gl.yformatter = LatitudeFormatter()

ax.spines['geo'].set_linewidth(0.3)
ax.spines['geo'].set_capstyle('butt')

plt.savefig(foldername + 'sensor_map_3.png',bbox_inches = 'tight', pad_inches=0, format='png',dpi=600)
plt.show()

In [ ]:
cmap2 = ListedColormap(["tab:orange", "tab:green"])

In [ ]:
sns.set(style='ticks', context='paper', font_scale=1.5)
fig = plt.figure(figsize = (6,6), dpi = DPI)
ax = plt.axes()
sensors_list['x [km]'] = sensors_list['CHY']/1000
sensors_list['y [km]'] = sensors_list['CHX']/1000
data = sensors_list.loc[sensors_list['region']=='Jura'] #Ticino, Napf
region = data['region'].iloc[0]
sns.scatterplot(ax=ax,x=data['x [km]'], y=data['y [km]'], c=cmap([2]), s = 40,
                edgecolor='black',linewidth=0.25)
plt.grid(True)
#ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.savefig(foldername + 'sensor_location'+ '_' + region + '.png',bbox_inches = 'tight', dpi=DPI)

## Figure 3

In [ ]:
def plot_event_time_series(ID,min_time):
    OUTPUT_FOLDER_TIME_SERIES = 'C:/.../day_sensor/'
    plot_data = sensors_filtered[sensors_filtered['day_sensor_ID']==ID]
    day = plot_data['days'].iloc[0]
    sensor = plot_data['sensorName'].iloc[0]
    region = plot_data['region'].iloc[0]
    n_hits = plot_data['daily_count_by_sensor'].iloc[0]
    fig = plt.figure(figsize = (13,4), dpi = DPI)
    ax = plt.axes()
    fs = 10
    sns.set(style='ticks', context='paper',font_scale=1.8)
    sns.scatterplot(x=plot_data['Time2'], y=plot_data['diameter'], color='red',s = 40, edgecolor='black', linewidth=0.4, ax=ax)

    majorFmt = mdates.DateFormatter('%H:%M')
    ax.xaxis.set_major_formatter(majorFmt)
    plt.title('')
    plt.suptitle('')
    plt.ylabel('diameter [mm]')
    plt.xlabel('Time UTC')
    plt.ylim(0,30)
    plt.yticks(np.arange(0, 35, step=5));
    plt.grid(True)
    
    plt.savefig(OUTPUT_FOLDER_TIME_SERIES + 'timeseries_' + day + '_' + region + '_' + sensor + '_' + str(n_hits) + '.png',bbox_inches = 'tight', dpi=DPI)

In [ ]:
%%capture
day_list = sensors_filtered.loc[sensors_filtered['daily_count_by_sensor'] >= 5, 'day_sensor_ID'].unique()
[plot_event_time_series(x,0) for x in day_list]

In [ ]:
plot_event_time_series(1,0)

# General observations, hailstones size and kinetic energy distributions

In [ ]:
sensors_filtered = make_sensors_filtered(reports_data, 'maxczc_rad4', 35)
sensors_filtered = sensors_filtered.loc[sensors_filtered['diameter']>=5]

In [ ]:
sensors_filtered['diameter'].describe()

In [ ]:
sensors_filtered['kin_energy'].describe()

In [ ]:
len(sensors_filtered[sensors_filtered['diameter']>=20])

In [ ]:
adj = 0.5
min_D = 5
D = np.arange(min_D, 35, 2*adj)
n, d = np.histogram(sensors_filtered['diameter'], bins = D, density=False)
n_norm = n/n.sum()

## Figure 4

In [ ]:
# Sensors data - Nb of measurements by month

nmeas_valid = []

startdate = '2018-07'
enddate = '2022-09'
months = pd.date_range(startdate,enddate,freq='M').strftime('%Y-%m')

for i,s in enumerate(months):
    nmeas_valid.append(np.sum(sensors_filtered[sensors_filtered['months'] == s]['status'] == 0))
                         
plt.figure(figsize = (10,4), dpi = DPI)
ax = plt.subplot()
plt.bar(range(len(months)),nmeas_valid)
plt.title('Number of sensor impacts per month')
plt.xticks(range(len(months)), months, rotation = 90, fontsize = 12)
min_loc = 1
max_loc = 2
ax.xaxis.set_major_locator(MultipleLocator(max_loc))
ax.xaxis.set_minor_locator(MultipleLocator(min_loc))
plt.margins(x=0.01, tight=True)
plt.savefig(foldername + 'sensors_barplot_months.png',bbox_inches = 'tight',dpi=DPI)

## Exponential fits

In [ ]:
def powerlaw(x, a, b, c):
    return a*np.power(x,b) + c

def powerlaw_2(x, a, b):
    return a*np.power(x,b)

def powerlaw_3(x, a, b):
    return a * np.exp(-b * x)

def powerlaw_4(x, a, b):
    return a * np.power(10, -b * x)

## Figure 5

In [ ]:
def plot_diam_dist_prob(data):
    sns.set(style='ticks', context='paper', font_scale=2)
    plt.figure(figsize = (10,5), dpi=300)
    ax = plt.axes()
    #sns.histplot(x = data['diameter'], kde=False, stat='count',bins=30, cumulative=False, ax=ax, color='blue')
    sns.histplot(x = data['diameter'], kde=False, stat='probability', binrange=(5,35), binwidth=1, cumulative=False, ax=ax, color='blue')
    ax.scatter(d[0:-1] + adj,n_norm,color='tab:red')
    popt, pcov = curve_fit(powerlaw_4, d[0:-1] + adj, n_norm)
    err_pcov = np.sqrt(np.diag(pcov))
    x_val = np.linspace(0,35,100)
    pw1 = np.append(popt,err_pcov)
    ax.plot(x_val, powerlaw_3(x_val, *popt), '-r', label='Exponential fit (eq. 3)')
    #ax.set_xscale('log')
    ax.set_yscale('log')
    plt.xlim(4,36)
    #plt.ylabel('# of impacts')
    plt.xlabel('Diameter [mm]')
    ax.legend()
    plt.savefig(folder_dt_analysis  + 'Diameter_prob_distribution_minD5.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
plot_diam_dist_prob(sensors_filtered)

In [ ]:
popt, pcov = curve_fit(powerlaw_4, d[0:-1] + adj, n_norm)
err_pcov = np.sqrt(np.diag(pcov))
pw1 = np.append(popt,err_pcov)
pw1

In [ ]:
def plot_kin_energy_dist(data):
    sns.set(style='ticks', context='paper', font_scale=2)
    plt.figure(figsize = (10,5), dpi=300)
    ax = plt.axes()
    sns.histplot(x = data['kin_energy'], kde=False, stat='count',bins=30,log_scale=True,cumulative=False, ax=ax, color='blue')
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.ylabel('# of impacts')
    plt.xlabel('Kinetic energy [J]')
    plt.savefig(folder_dt_analysis  + 'KE_distribution_' + '_CZC35_minD5.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
plot_kin_energy_dist(sensors_filtered)

# Comparison with hailpads data

In [ ]:
hailpads_path = 'C:../Hail Pads/'
hailpads_dist = 'manzato_hailpads_dist.csv'
hailstones_dist = 'manzato_hailstones_dist.csv'
hailpads_qt = 'hailpads_quantiles.csv'
f_hailpad = 'hailpads_comp/'

HAILPAD_MEAN = 8.030818
HAILPAD_SD = 2.203626
HAILPAD_N = 747757
HAILPAD_SURFACE = 0.115

hailpads_data = pd.read_csv(hailpads_path + hailpads_dist,sep=';')
hailpads_data = hailpads_data.rename(columns={'stones_hailpad':'Daily_hits_per_device'})
hailpads_data['Counts_norm'] = hailpads_data['Counts']/sum(hailpads_data['Counts'])
hailpads_data['hits/m2'] = hailpads_data['Daily_hits_per_device']/HAILPAD_SURFACE
hailpads_data['Device'] = 'Hailpads'

plot_data = pd.concat([hailpads_data,hail_days])

hailstones_data = pd.read_csv(hailpads_path + hailstones_dist,sep=';')
hailstones_data = hailstones_data.rename(columns={'HailSize_BinCenter':'Size bin center'})
hailstones_data = hailstones_data.rename(columns={'HailSize_Prob':'Probability'})
hailstones_data['Size bin center adj'] = hailstones_data['Size bin center'] + 0.05

qt_data = pd.read_csv(hailpads_path + hailpads_qt,sep=';')

In [ ]:
min_D = [5.95] #5.95
adj = 0.25 # 0.25

dens = False
field_th = [35]
field_name = ['maxczc_rad4']
temp = []
diam_dict = {}
diam_dist_dict = {}
D_list = []

for i,j in zip(field_th,field_name):
    for k in min_D:
        sensors_data = make_sensors_filtered(reports_data, j, i)
        sensors_data = sensors_data.loc[sensors_data['diameter']>=k]

        diam = sensors_data['diameter']
        D = np.arange(k, 47.45, 2*adj)
        n, x = np.histogram(diam, bins = D, density=dens)
        n = n/n.sum()
        D_list.append(D)

        df = sensors_data.groupby('day_sensor_ID')['daily_count_by_sensor'].agg('min').to_frame()
        df = df.groupby('daily_count_by_sensor')['daily_count_by_sensor'].count().to_frame()
        df = df.rename(columns={"daily_count_by_sensor":"Counts"}).reset_index()
        df = df.rename(columns={"daily_count_by_sensor":"Daily_hits_per_device"})
        df['Counts_norm'] = df['Counts']/sum(df['Counts'])
        df['hits/m2'] = df['Daily_hits_per_device']/SENSOR_SURFACE
        #df['device'] = 'hail sensor ' + j[3:6] + str(i) + '_' + str(k) + 'mm'
        df['Device'] = 'Hail sensors'
        temp.append(df)
        diam_dist_dict[df['Device'].iloc[0]] = n
        diam_dict[df['Device'].iloc[0]] = diam

hail_days = pd.concat(temp)

## Figure 6

In [ ]:
plot_data['Device'] = pd.Categorical(plot_data['Device'], ['Hail sensors','Hailpads'])

In [ ]:
def plot_devices_dist(data):
    sns.set(style='ticks', context='paper', font_scale=2)
    plt.figure(figsize = (10,5), dpi=300)
    ax = plt.axes()
    sns.histplot(x = data['Daily_hits_per_device'], weights = data['Counts'], kde=False, stat='probability', bins=50, cumulative=False, ax=ax, hue=data['Device'],multiple='dodge',common_norm=False)
    ax.set_yscale('log')
    plt.ylabel('Probability')
    plt.xlabel('Number of hailstones per device')
    plt.savefig(foldername + f_hailpad + 'Devices_distribution.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
plot_devices_dist(plot_data)

In [ ]:
def plot_devices_dist_zoom(data):
    sns.set(style='ticks', context='paper', font_scale=2)
    plt.figure(figsize = (10,5), dpi=300)
    ax = plt.axes()
    sns.histplot(x = data['Daily_hits_per_device'], weights = data['Counts'], kde=False, stat='probability', bins=1245, cumulative=False, ax=ax, hue=data['Device'],multiple='dodge',common_norm=False)
    #ax.set_yscale('log')
    plt.ylabel('Probability')
    plt.xlabel('Number of hailstones per device')
    plt.xlim(0,50)
    plt.savefig(foldername + f_hailpad + 'Devices_distribution_zoom.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
plot_devices_dist_zoom(plot_data)

## Figure 7

In [ ]:
popt, pcov = curve_fit(powerlaw_4,hailstones_data['Size bin center'].to_numpy(), hailstones_data['Probability'].to_numpy())
err_pcov = np.sqrt(np.diag(pcov))
pw = np.append(popt,err_pcov)
popt2, pcov2 = curve_fit(powerlaw_4, x[0:-1] + adj, n)
err_pcov2 = np.sqrt(np.diag(pcov2))
pw2 = np.append(popt2,err_pcov2)

In [ ]:
pw2

In [ ]:
pw

In [ ]:
sns.set(style='ticks', context='paper', font_scale=1.5)
plt.figure(figsize = (10,5), dpi=300)
ax = plt.axes()

for (key, value), d in zip(diam_dist_dict.items(), D_list):
    ax.semilogy(d[0:-1] + adj, value,lw=2, label=key)
    
ax.semilogy(hailstones_data['Size bin center'].to_numpy(), hailstones_data['Probability'].to_numpy(),lw=2,label="Hailpads")

x_val = np.linspace(5.95,47.45,100)
#

ax.plot(x_val, powerlaw_4(x_val, *popt2), 'r--', label='Fit hail sensors (eq. 4)')
ax.plot(x_val, powerlaw_4(x_val, *popt), 'g--', label='Fit hailpads (eq. 5)')


plt.ylabel('Probability')
plt.xlabel('Diameter [mm]')
plt.xticks(np.arange(0, 50, step=5));
plt.legend()
plt.savefig(foldername + f_hailpad + 'HSD_ground_hailpad_sensor.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
sns.set(style='ticks', context='paper', font_scale=1.5)
plt.figure(figsize = (10,5), dpi=300)
ax = plt.axes()

for (key, value), d in zip(diam_dist_dict.items(), D_list):
    ax.plot(d[0:-1] + adj, value,lw=2, label=key)

ax.plot(hailstones_data['Size bin center'].to_numpy(), hailstones_data['Probability'].to_numpy(),lw=2,label="Hailpads")
#x_val = np.linspace(5.95,47.45,100)
#
#ax.plot(x_val, powerlaw_4(x_val, *popt), '-r', label='Exponential fit (eq. 3)')
#ax.plot(x_val, powerlaw_4(x_val, *popt2), '-g', label='Exponential fit (eq. 3)')

plt.ylabel('Probability')
plt.xlabel('Diameter [mm]')
plt.xticks(np.arange(0, 25, step=2));
plt.xlim(5,20)
plt.legend()
plt.savefig(foldername + f_hailpad + 'HSD_ground_hailpad_sensor_lin_zoom.png', bbox_inches = 'tight', dpi=DPI)

## Statistical analysis

### Figure 8

In [ ]:
qt_dict = {}
for key, value in diam_dict.items():
    t = np.percentile(value, np.arange(1,101,1))
    qt_dict[key] = t

In [ ]:
def qq_plot(data,data_ref):
    sns.set(style='ticks', context='paper', font_scale=2)
    fig = plt.figure(figsize = (8,8), dpi=300)
    ax = plt.axes()
    for key, value in data.items():
        sns.scatterplot(x = value, y = data_ref['Dhail'], ax=ax, s=30, linewidth=0, label=key)
    sns.lineplot(x=data_ref['Dhail'], y = data_ref['Dhail'], color='red', ax=ax);
    plt.xlabel('Quantile of diameter, hail sensors')
    plt.ylabel('Quantile of diameter, hailpads')
    plt.xticks(np.arange(0, 50, step=5))
    plt.yticks(np.arange(0, 50, step=5))
    plt.xlim(5,47)
    plt.ylim(5,47)
    ax.get_legend().remove()
    plt.savefig(foldername + f_hailpad + 'HSD_QQplot.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
qq_plot(qt_dict,qt_data)

### Chi-squared test

In [ ]:
df_bins = pd.DataFrame()
#df_bins['hailpads_obs'] = hailstones_data['Probability'] * HAILPAD_N
#df_bins['hailpads_obs'] = df_bins['hailpads_obs'].round(0).astype(int)
name_list = []
bins = qt_data['Dhail'].to_numpy()#[9::10]

bins = np.insert(bins, 0, 5.95)

for key, value in diam_dict.items():
    df_bins[key] = pd.cut(value, bins=bins).value_counts().values
    name_list.append(key)

df_bins['hailpads_obs'] = HAILPAD_N/100
df_bins['hailpads_obs'] = df_bins['hailpads_obs'].round(0).astype(int)

In [ ]:
from scipy.stats import chisquare
data = df_bins.head(100).copy()
for f in name_list:
    key_exp = f + '_exp'
    data[key_exp] = data['hailpads_obs'] / np.sum(data['hailpads_obs']) * np.sum(data[f])
    stat, p_value = chisquare(data[f], data[key_exp])
    print(f"Chi-squared Test: statistic={stat:.4f}, p-value={p_value:.4f}")


### Standardized Mean Difference (SMD)

In [ ]:
smd_list = []
for key, value in diam_dict.items():
    m = np.mean(value)
    std = np.std(value)
    smd = abs(HAILPAD_MEAN - m)/np.sqrt((HAILPAD_SD**2 + std**2)/2)
    smd_list.append(smd)
print(f"SMD={smd_list[0]:.4e}")

### Kolmogorov-Smirnov test

In [ ]:
from scipy.stats import kstwo
for key, value in qt_dict.items():
    sample = qt_data['Dhail'].to_frame()
    sample['Group'] = 'hailpads'
    df = pd.DataFrame(value)
    df = df.rename(columns={0: "Dhail"})
    df['Group'] = 'hailsensors'
    df_all = pd.concat([sample,df])
    d_c = df_all.loc[df_all.Group=='hailpads', 'Dhail'].values
    d_t = df_all.loc[df_all.Group=='hailsensors', 'Dhail'].values
    df_ks = pd.DataFrame()
    df_ks['Dhail'] = np.sort(df_all['Dhail'].unique())
    df_ks['F_control'] = df_ks['Dhail'].apply(lambda x: np.mean(d_c<=x))
    df_ks['F_treatment'] = df_ks['Dhail'].apply(lambda x: np.mean(d_t<=x))
    k = np.argmax( np.abs(df_ks['F_control'] - df_ks['F_treatment']))
    ks_stat = np.abs(df_ks['F_treatment'][k] - df_ks['F_control'][k])
    m, n = len(sample), len(df)
    en = m * n / (m + n)
    p_value = kstwo.sf(ks_stat, np.round(en))
    print(f" Kolmogorov-Smirnov Test: statistic={ks_stat:.4f}, p-value={p_value:.4e}")

# Analysis of time-related quantities

In [ ]:
EVENT_MIN_TIME_LIST = [300,600,900,1200]
EVENT_MIN_TIME_NAME = ['5 min','10 min','15 min','20 min']

HIT_TH = 2
HIT_TH_UP = 1000
temp_events = []
temp_hits = []
field_name = 'maxczc_rad4'
field_th = 35
min_D = 5

sensors_filtered = make_sensors_filtered(reports_data, field_name, field_th)
sensors_filtered = sensors_filtered.loc[sensors_filtered['diameter']>=min_D]
for i,j in zip(EVENT_MIN_TIME_LIST,EVENT_MIN_TIME_NAME):
    sensors_data = make_sensors_data(sensors_filtered, i)
    hail_events_full = make_hail_events_full(sensors_data)
    hail_events_sel, sensors_ev_sel = make_sensors_sel(hail_events_full, sensors_data, HIT_TH, HIT_TH_UP)
    
    hail_events_sel['EVENT_MIN_TIME'] = i
    hail_events_sel['EVENT_MIN_TIME_NAME'] = j
    sensors_ev_sel['EVENT_MIN_TIME'] = i
    sensors_ev_sel['EVENT_MIN_TIME_NAME'] = j
    temp_events.append(hail_events_sel)
    temp_hits.append(sensors_ev_sel)

df_events = pd.concat(temp_events)
df_hits = pd.concat(temp_hits)
temp = df_events.groupby(['EVENT_MIN_TIME','Event_hits_group'])['Event_hits']
df_events = df_events.reset_index()
df_events = df_events.set_index(['EVENT_MIN_TIME','Event_hits_group'])
df_events['Event_hits_group_size'] = temp.size()
df_events['Event_hits_group_sum'] = temp.sum()
df_events['Event_duration_min'] = df_events['Event_duration']/60
df_events = df_events.reset_index()

### Dead time analysis

In [ ]:
df_events.groupby('EVENT_MIN_TIME')['Event_hits_diff2'].sum()/df_events.groupby('EVENT_MIN_TIME')['Event_hits'].sum()

## Data for Table 1

### Event duration

In [ ]:
df_events.groupby(['EVENT_MIN_TIME_NAME'])['Event_duration_min'].describe()

In [ ]:
df_events.groupby(['EVENT_MIN_TIME_NAME','Event_hits_group'])['Event_duration_min'].describe()

### Cumulative time distribution of impacts (CTDI)

In [ ]:
#Time0 in seconds, convert in minutes
df_hits.groupby(['EVENT_MIN_TIME_NAME'])['Time_0'].describe()

In [ ]:
#Time0 in seconds, convert in minutes
df_hits.groupby(['EVENT_MIN_TIME_NAME','Event_hits_group'])['Time_0'].describe()

## Figure 9

In [ ]:
def add_median_labels(ax, fmt='.1f'):
    lines = ax.get_lines()
    boxes = [c for c in ax.get_children() if type(c).__name__ == 'PathPatch']
    lines_per_box = int(len(lines) / len(boxes))
    for median in lines[4:len(lines):lines_per_box]:
        x, y = (data.mean() for data in median.get_data())
        # choose value depending on horizontal or vertical plot orientation
        value = x if (median.get_xdata()[1] - median.get_xdata()[0]) == 0 else y
        text = ax.text(x, y, f'{value:{fmt}}', ha='center', va='center',
                       fontweight='bold', color='white')
        # create median-colored border around white text for contrast
        text.set_path_effects([
            path_effects.Stroke(linewidth=3, foreground=median.get_color()),
            path_effects.Normal(),
        ])

In [ ]:
sns.set(style='ticks', context='paper', font_scale=1.5)
plt.figure(figsize = (10,7), dpi=200)
#plt.figure()
ax = plt.axes()

g = sns.boxplot(data=df_events, x="EVENT_MIN_TIME_NAME", y="Event_duration_min", hue="Event_hits_group",hue_order=['2-5 impacts','6-25 impacts','> 25 impacts'],showfliers = False,ax=ax,saturation=1,palette='Set1')
add_median_labels(g)
plt.ylabel('Event duration [min]')
plt.xlabel('$t_{mb}$ [min]')
new_title = 'Impacts per event'
leg = g.axes.get_legend()
leg.set_title(new_title)

plt.savefig(folder_dt_analysis + 'Boxplots_event_duration_CZC35_minD5_minutes.png',bbox_inches = 'tight',dpi=DPI)

## Figure 10

In [ ]:
colors = [sns.light_palette(sns.color_palette('Set1')[0],8)[1],sns.light_palette(sns.color_palette('Set1')[1],8)[1],sns.light_palette(sns.color_palette('Set1')[2],8)[1],
          sns.light_palette(sns.color_palette('Set1')[0],8)[3],sns.light_palette(sns.color_palette('Set1')[1],8)[3],sns.light_palette(sns.color_palette('Set1')[2],8)[3],
          sns.light_palette(sns.color_palette('Set1')[0],8)[5],sns.light_palette(sns.color_palette('Set1')[1],8)[5],sns.light_palette(sns.color_palette('Set1')[2],8)[5],
          sns.light_palette(sns.color_palette('Set1')[0],8)[7],sns.light_palette(sns.color_palette('Set1')[1],8)[7],sns.light_palette(sns.color_palette('Set1')[2],8)[7]]
customPalette = sns.set_palette(sns.color_palette(colors))

colors_2 = [sns.color_palette('Greys',8)[1],sns.color_palette('Greys',8)[3],sns.color_palette('Greys',8)[5],sns.color_palette('Greys',8)[7]]

In [ ]:
sns.set(style='ticks', context='paper', font_scale=1.5)
plt.figure(figsize = (10,7), dpi=200)
#plt.figure()
ax = plt.axes()

hue_order = ['$t_{mb}$: 5 min, 2-5 impacts','$t_{mb}$: 5 min, 6-25 impacts','$t_{mb}$: 5 min, > 25 impacts',
            '$t_{mb}$: 10 min, 2-5 impacts','$t_{mb}$: 10 min, 6-25 impacts','$t_{mb}$: 10 min, > 25 impacts',
            '$t_{mb}$: 15 min, 2-5 impacts','$t_{mb}$: 15 min, 6-25 impacts','$t_{mb}$: 15 min, > 25 impacts',
            '$t_{mb}$: 20 min, 2-5 impacts','$t_{mb}$: 20 min, 6-25 impacts','$t_{mb}$: 20 min, > 25 impacts']
hue = '$t_{mb}$: ' + df_hits['EVENT_MIN_TIME_NAME'] + ', ' + df_hits['Event_hits_group'].astype(str)
sns.ecdfplot(data=df_hits, x="Time_0", hue=hue, hue_order=hue_order,palette = sns.color_palette(colors))

plt.axhline(y=0.75, color='black', linestyle='-')

plt.xlim(-10,700)
plt.ylabel('Cumulative time distribution of impacts (CTDI)')
plt.xlabel('Time since event start [s]')

plt.savefig(folder_dt_analysis + 'Perc_hits_event_time_CZC35_minD5.png',bbox_inches = 'tight',dpi=DPI)

## Figure 11

In [ ]:
def plot_event_hits_hitrate_density(data,EVENT_MIN_TIME, field):
    data = data.sort_values([field], ascending=[True])
    cmap = sns.color_palette("flare", as_cmap=True)
    sns.set(style='ticks', context='paper', font_scale=1.5)
    plt.figure(figsize = (7,7), dpi=300)
    ax = plt.axes()
    ax.set(xscale="log", yscale="log")
    g = sns.scatterplot(x=data['Event_hit_rate'], y=data['Event_hits'], hue=data[field],size=data[field],ax=ax,sizes=(10, 100), palette=cmap, s=40)
    #sns.histplot(x = data['Event_hit_rate'], y = data['Event_hits'], kde=True, stat='count',cumulative=False, ax=ax, color='blue')
    #sns.relplot(x=data['Event_hit_rate'], y = data['Event_hits'], hue=data['Event_30s_hit_rate_max'], size=data['Event_30s_hit_rate_max'],
    #            sizes=(10, 100), palette=cmap, s=40)
    plt.ylabel('Number of impacts per event')
    plt.xlabel('Average hit rate [#/s]')
    plt.xlim(0.002,2)
    plt.ylim(1,405)
    new_title = 'Maximum \ninstanteanous \nhit rate [#/s]'
    leg = g.axes.get_legend()
    leg.set_title(new_title)
    # norm = plt.Normalize(data[field].min(), data[field].max())
    # sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    # sm.set_array([])
    # ax.get_legend().remove()
    # ax.figure.colorbar(sm)
    
    plt.savefig(folder_dt_analysis + 'Event_hits_hitrate_' + field + '_T' + str(EVENT_MIN_TIME) + '.png', bbox_inches = 'tight', dpi=DPI)

In [ ]:
plot_event_hits_hitrate_density(df_events[df_events['EVENT_MIN_TIME']==600],600,'Event_inst_hit_rate_max')

## Figure 12

In [ ]:
sns.set(style='ticks', context='paper', font_scale=1.5)
plt.figure(figsize = (10,7), dpi=200)
#plt.figure()
ax = plt.axes()

data = df_events.loc[df_events['Event_hits_group'].isin(['6-25 impacts','> 25 impacts'])]

cm1 = [sns.color_palette('Set1',3)[1],sns.color_palette('Set1',3)[2]]

g = sns.boxplot(data=data, x="EVENT_MIN_TIME_NAME", y="Time_rel", hue="Event_hits_group",hue_order=['6-25 impacts','> 25 impacts'],
                showfliers = False,ax=ax,saturation=1,palette=cm1)
add_median_labels(g,'.2f')

plt.ylabel('Relative time of maximum diameter [s]')
plt.xlabel('$t_{mb}$ [s]')
new_title = 'Impacts per event'
leg = g.axes.get_legend()
leg.set_title(new_title)

plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)

plt.savefig(folder_dt_analysis + 'Boxplots_DmaxReltime_CZC35_minD5.png',bbox_inches = 'tight',dpi=DPI)